In [25]:
%matplotlib inline
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import LSTM, Convolution1D, Flatten, Dropout, MaxPooling1D, Dense, Merge
# from keras.layers.convolution import 
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard
import ast
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import keras as kr

from keras.callbacks import ModelCheckpoint
from sklearn.utils import class_weight
from keras.optimizers import SGD,adam

from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
print(kr.__version__,tf.__version__)

2.0.4 1.1.0


In [26]:
base_dir_tr='training/'
base_dir_vl='validation/'
weight_log='weight_logs/'

In [27]:
top_words = 20000
dtr_1=pd.read_csv(base_dir_tr+'data_manipulation_2_2_train.csv',encoding='ISO-8859-1')
dtv_1=pd.read_csv(base_dir_vl+'data_manipulation_2_2_valid.csv',encoding='ISO-8859-1')
dtr_2=pd.read_csv(base_dir_tr+'data_cat_tr.csv',encoding='ISO-8859-1')
dtv_2=pd.read_csv(base_dir_vl+'data_cat_vl.csv',encoding='ISO-8859-1')

In [28]:
#### category part
dcat_l=[]
for cc in range(3):
    dum=pd.get_dummies(dtr_2[dtr_2.columns[cc+1]])
    col_i=[dtr_2.columns[cc+1]+'_'+str(i) for i in dum.columns]
    dum.columns=col_i
    dcat_l.append(dum)
dcat_tr_1=pd.concat(dcat_l,axis=1)
Xcat_tr=dcat_tr_1.as_matrix()

dcat_l=[]
for cc in range(3):
    dum=pd.get_dummies(dtv_2[dtv_2.columns[cc+1]])
    col_i=[dtv_2.columns[cc+1]+'_'+str(i) for i in dum.columns]
    dum.columns=col_i
    dcat_l.append(dum)
dcat_tv_1=pd.concat(dcat_l,axis=1)
Xcat_tv=dcat_tv_1.as_matrix()

print(len(dcat_tr_1),len(dcat_tv_1),len(dcat_tv_1.columns))

36283 11838 251


In [29]:
Xsd=dtr_1.rank_short_des.as_matrix()
Xsd=np.array([ast.literal_eval(cd) for cd in Xsd])

Xti=dtr_1.rank_title.as_matrix()
Xti=np.array([ast.literal_eval(cd) for cd in Xti])

ycl=dtr_1.clari

In [30]:
Xsd_v=dtv_1.rank_short_des.as_matrix()
Xsd_v=np.array([ast.literal_eval(cd) for cd in Xsd_v])

Xti_v=dtv_1.rank_title.as_matrix()
Xti_v=np.array([ast.literal_eval(cd) for cd in Xti_v])

In [31]:
# L_Xsd=[len(x) for x in Xsd]
# L_Xti=[len(x) for x in Xti]

# dLsd=pd.DataFrame(L_Xsd)
# dLti=pd.DataFrame(L_Xti)

# dLsd[dLsd<200].hist(bins=50)

In [32]:
max_review_length_s = 150
max_review_length_t = 40
Xsd = sequence.pad_sequences(Xsd, maxlen=max_review_length_s)
Xti = sequence.pad_sequences(Xti, maxlen=max_review_length_t)

Xsd_v = sequence.pad_sequences(Xsd_v, maxlen=max_review_length_s)
Xti_v = sequence.pad_sequences(Xti_v, maxlen=max_review_length_t)

In [33]:
X=[]
for i,j,k in zip(Xsd,Xti,Xcat_tr):
    X.append(list(k)+list(j)+list(i))
X=np.array(X)

X_v=[]
for i,j,k in zip(Xsd_v,Xti_v,Xcat_tv):
    X_v.append(list(k)+list(j)+list(i))
X_v=np.array(X_v)

# np.savetxt(base_dir_vl+'data_input_ycl_3_1.txt',X_v)
# np.savetxt(base_dir_vl+'data_input_ycl_4_1.txt',X_v)
np.savetxt(base_dir_vl+'data_input_ycl_4_2.txt',X_v)

In [34]:
len(X[0][251:])==len(np.array(list(Xti[0])+list(Xsd[0])))

True

In [35]:
len(X[0][:251])==len(np.array(list(Xcat_tr[0])))

True

#### ycl model

In [36]:
# model.save('model_yco_1_2_3.h5') OKE (74.05%)
ycl.value_counts()/ycl.value_counts().sum()

1    0.943362
0    0.056638
Name: clari, dtype: float64

In [37]:
clas_weight = class_weight.compute_class_weight('balanced', np.unique(ycl), ycl)
clas_weight

array([ 8.82798054,  0.53001928])

In [38]:
X_train, X_test, ycl_train, ycl_test = train_test_split(X, ycl, test_size=0.3, random_state=21)

X_train_nl=np.array([x[251:] for x in X_train])
X_train_ct=np.array([x[:251] for x in X_train])

X_test_nl=np.array([x[251:] for x in X_test])
X_test_ct=np.array([x[:251] for x in X_test])

In [39]:
ycl_test.value_counts()/ycl_test.value_counts().sum()

1    0.94497
0    0.05503
Name: clari, dtype: float64

In [40]:
# len(X_train_nl)
X_train

array([[    0,     0,     0, ...,    50,   135,  5013],
       [    0,     0,     0, ...,   173,     6,   250],
       [    0,     0,     0, ...,  4831,    21,   641],
       ..., 
       [    0,     0,     0, ...,  1535,   233,   117],
       [    0,     0,     0, ...,   962,    18, 14122],
       [    1,     0,     0, ...,  2551,  3305,     0]], dtype=int32)

In [41]:
X_train_nl

array([[    0,     0,     0, ...,    50,   135,  5013],
       [    0,     0,     0, ...,   173,     6,   250],
       [    0,     0,     0, ...,  4831,    21,   641],
       ..., 
       [    0,     0,     0, ...,  1535,   233,   117],
       [    0,     0,     0, ...,   962,    18, 14122],
       [    0,     0,     0, ...,  2551,  3305,     0]], dtype=int32)

In [46]:
np.random.seed(21)
#### Creating the model_1
embedding_vecor_length = 150
model_nl = Sequential()
model_nl.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length_s+max_review_length_t))
# model_nl.add(Convolution1D(64, 4, border_mode='same'))
model_nl.add(Convolution1D(128, 4, border_mode='same'))
model_nl.add(Convolution1D(32, 4, border_mode='same'))
# model_nl.add(Convolution1D(16, 4, border_mode='same'))
# model_nl.add(Convolution1D(16, 4, border_mode='same'))
model_nl.add(Flatten())
# model_nl.add(LSTM(80,activation='tanh', recurrent_activation='hard_sigmoid'))
model_nl.add(Dropout(0.2))
model_nl.add(Dense(200,activation='relu'))
model_nl.add(Dense(200,activation='relu'))
model_nl.add(Dense(200,activation='relu'))
# model_nl.add(Dense(150,activation='relu'))
model_nl.add(Dropout(0.1))
model_nl.add(Dense(1,activation='relu'))

#### Creating the model_2
model_ct = Sequential()
model_ct.add(Dense(70, input_shape=(251,),activation='relu'))
model_ct.add(Dense(70, input_shape=(251,),activation='relu'))
model_ct.add(Dense(70, input_shape=(251,),activation='relu'))
model_ct.add(Dense(35, input_shape=(251,),activation='relu'))
model_ct.add(Dense(35, input_shape=(251,),activation='relu'))
model_ct.add(Dense(35, input_shape=(251,),activation='relu'))
model_ct.add(Dense(17, input_shape=(251,),activation='relu'))
# model_ct.add(Dense(12, input_shape=(251,),activation='relu'))

model_ct.add(Dropout(0.0005))
model_ct.add(Dense(2,activation='relu'))

model = Sequential()
model.add(Merge([model_ct,model_nl], mode='concat'))
model.add(Dense(1,activation='sigmoid'))

model2=model

# opti = SGD(lr=0.6)
opti = adam(lr=0.0008) #0.001
# model.compile(loss = "categorical_crossentropy", optimizer = opt)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','mse'])
model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy','mse'])

# model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy','mse'])


# checkpoint
# filepath=weight_log+"weights_best_ycl_3_1.hdf5"
filepath=weight_log+"weights_best_ycl_4_2.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_mean_squared_error', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 4, padding="same")`
/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 4, padding="same")`
/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:36: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [47]:
model.input_shape

[(None, 251), (None, 190)]

In [48]:
print(X_train_ct.shape,X_train_nl.shape)

(25398, 251) (25398, 190)


In [49]:
model.fit([X_train_ct,X_train_nl],ycl_train, class_weight=clas_weight, validation_data=([X_test_ct,X_test_nl],ycl_test),callbacks=callbacks_list, epochs=200, batch_size=100, verbose=1)

Train on 25398 samples, validate on 10885 samples
Epoch 1/200
25398/25398 [==============================] - 95s - loss: 0.1959 - acc: 0.9415 - mean_squared_error: 0.0526 - val_loss: 0.1659 - val_acc: 0.9450 - val_mean_squared_error: 0.0449
Epoch 2/200
25398/25398 [==============================] - 91s - loss: 0.1476 - acc: 0.9427 - mean_squared_error: 0.0423 - val_loss: 0.1868 - val_acc: 0.9450 - val_mean_squared_error: 0.0469
Epoch 3/200
 8600/25398 [=========>....................] - ETA: 54s - loss: 0.1114 - acc: 0.9451 - mean_squared_error: 0.0335

KeyboardInterrupt: 

In [50]:
# model2.load_weights(weight_log+"weights_best_ycl_3_1.hdf5")
# model2.load_weights(weight_log+"weights_best_ycl_4_1.hdf5")
model2.load_weights(weight_log+"weights_best_ycl_4_2.hdf5")
model2.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy','mse'])

In [51]:
#### Evaluation on the test set
ycl_predc=model2.predict_classes([X_test_ct,X_test_nl])
ycl_predp=model2.predict([X_test_ct,X_test_nl])

ycl_predc=[i[0] for i in ycl_predc]
ycl_predp=[i[0] for i in ycl_predp]

scores = model2.evaluate([X_test_ct,X_test_nl], ycl_test, verbose=0)
mse_c=mse(ycl_test,ycl_predc)
mse_p=mse(ycl_test,ycl_predp)
print("Accuracy: %.2f%%" % (scores[1]*100)) 
print("MSE_class : ",mse_c)
print("MSE_prob : ",mse_p)

10880/10885 [============================>.] - ETA: 0sAccuracy: 94.50%
MSE_class :  0.0550298576022
MSE_prob :  0.0449408995951


In [52]:
# model_nl.save('model_ycl_4_1(1).h5') #<-0.45227
# model_ct.save('model_ycl_4_1(2).h5')

model_nl.save('model_ycl_4_2(1).h5') #<-0.04494
model_ct.save('model_ycl_4_2(2).h5')

### Validation

In [53]:
X_vl=np.loadtxt(base_dir_vl+'data_input_ycl_4_2.txt')

ycl_model_1=load_model('model_ycl_4_2(1).h5')
ycl_model_2=load_model('model_ycl_4_2(2).h5')
ycl_model = Sequential()
ycl_model.add(Merge([ycl_model_2,ycl_model_1], mode='concat'))
ycl_model.add(Dense(1,activation='sigmoid'))

ycl_model.load_weights(weight_log+"weights_best_ycl_4_2.hdf5")
ycl_model.compile(loss='binary_crossentropy', optimizer=opti, metrics=['accuracy','mse'])

/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/keras/models.py:248: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/home/dataanalysis/anaconda2/envs/python3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [54]:
X_vl_nl=np.array([x[251:] for x in X_vl])
X_vl_ct=np.array([x[:251] for x in X_vl])

In [55]:
ycl_predp=ycl_model.predict([X_vl_ct,X_vl_nl])
ycl_predp=[i[0] for i in ycl_predp]

In [56]:
with open('clarity_valid.predict','w') as fcl:
    for pp in ycl_predp:
        fcl.write(str(pp))
        fcl.write('\n')
fcl.close()